# Final Project

In [4]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
fname = "business.csv"
business_df = pd.read_csv(fname)

In [6]:
business_df

,business_owner_name,ownership_type,account_key,expiration_dt,doing_bus_as_name,address_str,apt_suite,city,state,zip,naics_description,po_box,pmb_box,creation_dt,bus_start_dt,naics_code,business_phone,lat,lon
0,moving team llc,llc,2017006233,2018-05-31,locksmith sd,2190 abbott st,NaN,san diego,CA,92107-2028,locksmiths,NaN,NaN,2017-05-30,2017-05-30,561622,(619) 930-0915,32.752988,-117.248651
1,jordan alimena,sole,2017006232,2018-05-31,direct apparel,9463 slope st,NaN,santee,CA,92071-4122,retail sales via internet/websites,NaN,NaN,2017-05-30,2017-05-30,454112,NaN,32.826873,-116.990471
2,alejandro m loya,sole,2017006231,2018-05-31,silva clean team,620 toyne st,NaN,san diego,CA,92102-4640,other personal services,NaN,NaN,2017-05-30,2017-05-22,8129,(619) 766-5404,32.712023,-117.103995
3,alyson rodeheaver,sole,2017006227,2018-04-30,alyson rodeheaver,4763 newport ave,NaN,san diego,CA,92107-2204,rental & leasing services,NaN,NaN,2017-05-30,2017-04-13,532,(619) 889-5427,32.744098,-117.247184
4,rhd usa inc,scorp,2017006226,2018-03-31,rhd usa inc,4345 twain ave,f,san diego,CA,92120-3437,miscellaneous store retailers,NaN,NaN,2017-05-30,2017-03-20,453,(619) 852-2939,32.784796,-117.100269
5,fournier foods llc,llc,2017006224,2018-09-30,the french oven bakery,10299 scripps trl,NaN,san diego,CA,92131-2366,retail bakeries,NaN,NaN,2017-05-30,2017-10-01,311811,(619) 857-0040,32.912540,-117.079360
6,lindsay gagnon,sole,2017006223,2018-02-28,just do fit,771 jamacha rd,316,el cajon,CA,92019-3202,personal fitness trainer,NaN,NaN,2017-05-30,2017-02-24,611621,(619) 962-7622,32.786118,-116.927187
7,rosalva gomez & maria m rivera,partnr,2017006219,2018-05-31,rossmary spa,4184 beyer blvd,207,san ysidro,CA,92173-2185,beauty shops - booth rental,NaN,NaN,2017-05-30,2017-06-01,812115,(619) 636-7795,32.559017,-117.044851
8,c & l refrigeration corporation,corp,2017006213,2018-05-31,c & l refrigeration corporation,479 nibus,NaN,brea,CA,92821-3204,"plumbing, heating & ac contractor",NaN,NaN,2017-05-30,2017-05-31,23511,(714) 990-6966,33.927389,-117.901267
9,attic perfect inc,corp,2017006211,2018-05-31,attic perfect inc,1001 hensley st,a,richmond,CA,94801-2108,masonry/drywall/insulation/tile contractors,NaN,NaN,2017-05-30,2017-05-31,2354,(800) 774-1740,37.946341,-122.366645


In [25]:
business_df = business_df[['doing_bus_as_name', 'address_str', 'zip','naics_description','lat','lon']]
business_df.loc[business_df['naics_description'] == 'full-service restaurants']

,doing_bus_as_name,address_str,zip,naics_description,lat,lon
17,jalapeno taco shop,3130 university ave,92104-2047,full-service restaurants,32.748683,-117.126968
55,awash ethiopian restaurant,2104 el cajon blvd,92104-1102,full-service restaurants,32.677079,-117.107167
79,the fire spot,4646 convoy st,92111-2315,full-service restaurants,32.773069,-117.156144
103,cold beers & cheeseburgers,322 05th ave,92101-6910,full-service restaurants,32.708791,-117.160357
233,broken yolk cafe,2642 del mar heights rd,92014-3110,full-service restaurants,32.764144,-117.119582
238,saigon on fifth,3900 05th ave,92103-3121,full-service restaurants,33.999810,-117.346091
308,spice & soul,4033 avati dr,92117-4403,full-service restaurants,32.814068,-117.216603
326,sushi diner 2,10330 friars rd,92120-2300,full-service restaurants,32.617338,-117.027080
340,waypoint public del sur,16480 paseo del sur,92127-4201,full-service restaurants,32.728034,-117.162257
382,el patron traditional mexican grill inc,9460 mira mesa blvd,92126-4871,full-service restaurants,32.752660,-117.196907
